# Import SOCCOM BGC-Argo floats

Using package ArgoPY to import as xarray dataset

*Note, this only needs to be run if the dataset hasn't already been downloaded*

In [1]:
import os                                            # open folders
import re                                            # regex to keep only numeric values in filename string
import pandas as pd                                  # for converting datetime
import numpy  as np                                  # for linspec
from   datetime              import date             # for saving figures with today's date
import gsw                                           # to calculate depth from pressure

# To get Argo data
from argopy import DataFetcher as ArgoDataFetcher
argo_loader = ArgoDataFetcher()

## Select WMOs corresponding to SOCCOM BGC-Argo floats

This is done by selected file names already downloaded in one of my folders, but could be improved to be more usable for others.

In [2]:
directory_with_data  = '/Users/hannah/Documents/UW-PMEL/Research/so_co2_flux_repo/data/01_raw/SOCCOM_bgc_argo_float_data/SOCCOM_HiResQC_LIAR_21Dec2021_netcdf/'
save_directory       = '/Users/hannah/Documents/UW-PMEL/Research/so_co2_flux_repo/data/02_intermediate/SOCCOM_bgc_argo_float_data/'  
directory            = os.fsencode(directory_with_data)


# # this set has questionaly good data
# questionably_good = np.array([5904677, 5905991, 5905635, 5905375, 5905378, 5905636, 5906007, 5906031, 5906030, 
#                               5906002, 5906208, 5906207, 5906215, 5906206, 5906226, 5906222, 5906211, 5906210])
# # corresponding to the following float numbers:
# # quest_good = [9631; 12688; 12701; 12709; 12748; 12754; 12880; 12885; 12888; 12889; 17898; 18013; 18098; 18821; 18829; 18852; 18864; 18994]


### Loop through filenames to get floats to add in:
bgc_argo_float_WMOs = []

for file in os.listdir(directory):  # for all files in this directory
    
    filename = os.fsdecode(file)

    if filename.endswith(".nc"):  # only the netcdf files
        if not filename.startswith('NO_WMO'):   # this is to not select files that don't have a WMO (only 2), but if we can figure out how to keep them, that would be nice!
        
            # Select just the WMO
            numeric_portion = int(re.findall(r'\d+', filename)[0])

            # Add WMO to list
            bgc_argo_float_WMOs.append(numeric_portion)


## Retreive Argo float data corresponding to selected WMOs

Note that this step takes somewhere around 20-30 minutes

In [6]:
# # commented out so that this isn't accidentally run
# soccom_bgc_argo_ds_points = argo_loader.float(bgc_argo_float_WMOs).to_xarray()


# # Save original SOCCOM BGC Argo dataset:
# savedate = date.today().strftime('%Y%m%d')
# soccom_bgc_argo_ds_points.to_netcdf(save_directory + 'soccom_bgc_argo_ds_points.nc'.format('.nc'))
# soccom_bgc_argo_ds_points.to_netcdf(save_directory + 'soccom_bgc_argo_ds_points_' + savedate + '.nc'.format('.nc'))

# soccom_bgc_argo_ds_points

<xarray.Dataset>
Dimensions:                (N_POINTS: 16942151)
Coordinates:
  * N_POINTS               (N_POINTS) int64 0 1 2 ... 16942148 16942149 16942150
    LATITUDE               (N_POINTS) float64 31.69 31.69 ... -48.49 -48.49
    LONGITUDE              (N_POINTS) float64 -64.2 -64.2 ... -143.3 -143.3
    TIME                   (N_POINTS) datetime64[ns] 2009-10-12T13:39:09 ... ...
Data variables: (12/13)
    CONFIG_MISSION_NUMBER  (N_POINTS) int32 1 1 1 1 1 1 1 ... 79 79 79 79 79 79
    CYCLE_NUMBER           (N_POINTS) int32 1 1 1 1 1 1 1 ... 79 79 79 79 79 79
    DATA_MODE              (N_POINTS) <U1 'D' 'D' 'D' 'D' ... 'D' 'D' 'D' 'D'
    DIRECTION              (N_POINTS) <U1 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_POINTS) int32 1901378 1901378 ... 5904186 5904186
    POSITION_QC            (N_POINTS) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ...                     ...
    PRES_QC                (N_POINTS) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    PSAL                   (N_POINTS) float64 36.17 36.17 36.17 ... 34.48 34.52
    PSAL_QC                (N_POINTS) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TEMP                   (N_POINTS) float64 26.17 26.16 26.16 ... 2.935 2.784
    TEMP_QC                (N_POINTS) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TIME_QC                (N_POINTS) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

## Convert points to profiles using argopy function

Note that this step takes ~5 minutes

In [31]:
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_points.argo.point2profile()


# Save original SOCCOM BGC Argo dataset:
savedate = date.today().strftime('%Y%m%d')
soccom_bgc_argo_ds_profiles.to_netcdf(save_directory + 'soccom_bgc_argo_ds_profiles.nc'.format('.nc'))
soccom_bgc_argo_ds_profiles.to_netcdf(save_directory + 'soccom_bgc_argo_ds_profiles_' + savedate + '.nc'.format('.nc'))

soccom_bgc_argo_ds_profiles

<xarray.Dataset>
Dimensions:                (N_PROF: 33547, N_LEVELS: 991)
Coordinates:
  * N_PROF                 (N_PROF) int64 754 755 756 757 ... 32513 33021 28724
  * N_LEVELS               (N_LEVELS) int64 0 1 2 3 4 5 ... 986 987 988 989 990
    LATITUDE               (N_PROF) float64 22.87 22.9 22.93 ... 34.07 4.453
    LONGITUDE              (N_PROF) float64 -158.0 -158.0 ... -60.06 -115.9
    TIME                   (N_PROF) datetime64[ns] 2007-12-23T10:58:56 ... 20...
Data variables: (12/13)
    CONFIG_MISSION_NUMBER  (N_PROF) int32 1 1 1 1 1 1 1 ... 89 75 90 63 56 219
    CYCLE_NUMBER           (N_PROF) int32 1 2 3 4 5 6 7 ... 89 75 90 63 56 219
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'D' 'D' ... 'A' 'A' 'A' 'A'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int32 5901468 5901468 ... 5906342 5906045
    POSITION_QC            (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ...                     ...
    PRES_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    PSAL                   (N_PROF, N_LEVELS) float64 34.99 34.99 ... nan nan
    PSAL_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TEMP                   (N_PROF, N_LEVELS) float64 24.47 24.47 ... nan nan
    TEMP_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TIME_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

In [73]:
# soccom_bgc_argo_ds_profiles_saved = soccom_bgc_argo_ds_profiles
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles_saved

In [34]:
soccom_bgc_argo_ds_profiles

<xarray.Dataset>
Dimensions:                (N_PROF: 33547, N_LEVELS: 991)
Coordinates:
  * N_PROF                 (N_PROF) int64 754 755 756 757 ... 32513 33021 28724
  * N_LEVELS               (N_LEVELS) int64 0 1 2 3 4 5 ... 986 987 988 989 990
    LATITUDE               (N_PROF) float64 22.87 22.9 22.93 ... 34.07 4.453
    LONGITUDE              (N_PROF) float64 -158.0 -158.0 ... -60.06 -115.9
    TIME                   (N_PROF) datetime64[ns] 2007-12-23T10:58:56 ... 20...
Data variables: (12/13)
    CONFIG_MISSION_NUMBER  (N_PROF) int32 1 1 1 1 1 1 1 ... 89 75 90 63 56 219
    CYCLE_NUMBER           (N_PROF) int32 1 2 3 4 5 6 7 ... 89 75 90 63 56 219
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'D' 'D' ... 'A' 'A' 'A' 'A'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int32 5901468 5901468 ... 5906342 5906045
    POSITION_QC            (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ...                     ...
    PRES_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    PSAL                   (N_PROF, N_LEVELS) float64 34.99 34.99 ... nan nan
    PSAL_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TEMP                   (N_PROF, N_LEVELS) float64 24.47 24.47 ... nan nan
    TEMP_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TIME_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

*Note: before interpolating data, check for any flags that are >2, indicating data with problems. During interpolation, flags are dropped. Currently only PSAL_QC has a 2 flag, which is still okay data.*

## Calculate depth from pressure

In [74]:
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(depth  = -1 * gsw.conversions.z_from_p(p = soccom_bgc_argo_ds_profiles.PRES, lat = soccom_bgc_argo_ds_profiles.LATITUDE))

In [37]:
soccom_bgc_argo_ds_profiles

<xarray.Dataset>
Dimensions:                (N_PROF: 33547, N_LEVELS: 991)
Coordinates:
  * N_PROF                 (N_PROF) int64 754 755 756 757 ... 32513 33021 28724
  * N_LEVELS               (N_LEVELS) int64 0 1 2 3 4 5 ... 986 987 988 989 990
    LATITUDE               (N_PROF) float64 22.87 22.9 22.93 ... 34.07 4.453
    LONGITUDE              (N_PROF) float64 -158.0 -158.0 ... -60.06 -115.9
    TIME                   (N_PROF) datetime64[ns] 2007-12-23T10:58:56 ... 20...
Data variables: (12/14)
    CONFIG_MISSION_NUMBER  (N_PROF) int32 1 1 1 1 1 1 1 ... 89 75 90 63 56 219
    CYCLE_NUMBER           (N_PROF) int32 1 2 3 4 5 6 7 ... 89 75 90 63 56 219
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'D' 'D' ... 'A' 'A' 'A' 'A'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int32 5901468 5901468 ... 5906342 5906045
    POSITION_QC            (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ...                     ...
    PSAL                   (N_PROF, N_LEVELS) float64 34.99 34.99 ... nan nan
    PSAL_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TEMP                   (N_PROF, N_LEVELS) float64 24.47 24.47 ... nan nan
    TEMP_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TIME_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    depth                  (N_PROF, N_LEVELS) float64 3.925 5.614 ... nan nan
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

## Interpolate data onto standard pressure levels

Note that these levels can be changed. They were selected to match what Dan(i) chose for the Weddell Gyre clusters

In [75]:
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.argo.interp_std_levels(np.linspace(20, 2000, 100))
# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(depth_interpolated  = -1 * gsw.conversions.z_from_p(p = soccom_bgc_argo_ds_profiles.PRES_INTERPOLATED, lat = soccom_bgc_argo_ds_profiles.lat))

In [76]:
soccom_bgc_argo_ds_profiles

<xarray.Dataset>
Dimensions:                (N_PROF: 1448, PRES_INTERPOLATED: 100)
Coordinates:
  * N_PROF                 (N_PROF) int64 12832 12835 12838 ... 752 27082 31810
    LATITUDE               (N_PROF) float64 52.73 52.72 52.53 ... -47.58 0.264
    LONGITUDE              (N_PROF) float64 -32.02 -30.6 -28.33 ... 106.1 -168.8
    TIME                   (N_PROF) datetime64[ns] 2014-03-01T19:29:42 ... 20...
  * PRES_INTERPOLATED      (PRES_INTERPOLATED) float64 20.0 40.0 ... 2e+03
Data variables:
    CONFIG_MISSION_NUMBER  (N_PROF) int32 10 13 16 21 7 28 ... 68 50 41 123 86
    CYCLE_NUMBER           (N_PROF) int32 10 13 16 21 7 28 ... 68 50 41 123 86
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'D' 'D' ... 'A' 'A' 'A' 'A'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int32 5904479 5904479 ... 5906032 5906300
    PRES                   (N_PROF, PRES_INTERPOLATED) float64 20.0 ... 2e+03
    PSAL                   (N_PROF, PRES_INTERPOLATED) float64 34.77 ... 34.64
    TEMP                   (N_PROF, PRES_INTERPOLATED) float64 5.196 ... 2.283
    depth                  (N_PROF, PRES_INTERPOLATED) float64 19.82 ... 1.98...
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

## Rename variables

In [77]:
# Rename dataset 'profiles_antarctic' for changes that are needed for GMM code 
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.rename({'N_PROF'            : 'profile', 
                                                                  'LATITUDE'          : 'lat', 
                                                                  'LONGITUDE'         : 'lon',
                                                                  'TIME'              : 'datetime',
                                                                  'PRES_INTERPOLATED' : 'pressure_interp',
                                                                  'PRES'              : 'pressure',
                                                                  'PSAL'              : 'practical_salinity',
                                                                  'TEMP'              : 'temperature'})

In [42]:
soccom_bgc_argo_ds_profiles

<xarray.Dataset>
Dimensions:                (profile: 1448, pressure_interp: 100)
Coordinates:
  * profile                (profile) int64 12832 12835 12838 ... 752 27082 31810
    lat                    (profile) float64 52.73 52.72 52.53 ... -47.58 0.264
    lon                    (profile) float64 -32.02 -30.6 ... 106.1 -168.8
    datetime               (profile) datetime64[ns] 2014-03-01T19:29:42 ... 2...
  * pressure_interp        (pressure_interp) float64 20.0 40.0 ... 2e+03
Data variables:
    CONFIG_MISSION_NUMBER  (profile) int32 10 13 16 21 7 28 ... 68 50 41 123 86
    CYCLE_NUMBER           (profile) int32 10 13 16 21 7 28 ... 68 50 41 123 86
    DATA_MODE              (profile) <U1 'D' 'D' 'D' 'D' 'D' ... 'A' 'A' 'A' 'A'
    DIRECTION              (profile) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (profile) int32 5904479 5904479 ... 5906032 5906300
    pressure               (profile, pressure_interp) float64 20.0 ... 2e+03
    practical_salinity     (profile, pressure_interp) float64 34.77 ... 34.64
    temperature            (profile, pressure_interp) float64 5.196 ... 2.283
    depth                  (profile, pressure_interp) float64 19.82 ... 1.98e+03
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

## Add year, month, day arrays to dataset

In [80]:
# Year
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(temp  = soccom_bgc_argo_ds_profiles.datetime)
soccom_bgc_argo_ds_profiles.temp.values = pd.to_datetime(soccom_bgc_argo_ds_profiles.temp.values).year
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.rename({'temp' : 'year'})

# Month
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(temp  = soccom_bgc_argo_ds_profiles.datetime)
soccom_bgc_argo_ds_profiles.temp.values = pd.to_datetime(soccom_bgc_argo_ds_profiles.temp.values).month
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.rename({'temp' : 'month'})

# Day
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(temp  = soccom_bgc_argo_ds_profiles.datetime)
soccom_bgc_argo_ds_profiles.temp.values = pd.to_datetime(soccom_bgc_argo_ds_profiles.temp.values).day
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.rename({'temp' : 'day'})

# # Season
# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(temp  = soccom_bgc_argo_ds_profiles.datetime)
# soccom_bgc_argo_ds_profiles.temp.values = xr.where(soccom_bgc_argo_ds_profiles.month)
# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.rename({'temp' : 'day'})


# Date: YYYYMMDD
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(temp  = soccom_bgc_argo_ds_profiles.datetime)
soccom_bgc_argo_ds_profiles.temp.values = pd.to_datetime(soccom_bgc_argo_ds_profiles.temp.values).strftime("%Y%m%d")
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.rename({'temp' : 'prof_YYYYMMDD'})

# Time: HHMMSS
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(temp  = soccom_bgc_argo_ds_profiles.datetime)
soccom_bgc_argo_ds_profiles.temp.values = pd.to_datetime(soccom_bgc_argo_ds_profiles.datetime.values).strftime("%H%M%S")
soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.rename({'temp' : 'prof_HHMMSS'})


# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(year  = pd.to_datetime(soccom_bgc_argo_ds_profiles.datetime.values).year)
# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(month = pd.to_datetime(soccom_bgc_argo_ds_profiles.datetime.values).month)
# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(day   = pd.to_datetime(soccom_bgc_argo_ds_profiles.datetime.values).day)

# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(prof_YYYYMMDD = pd.to_datetime(soccom_bgc_argo_ds_profiles.datetime.values).strftime("%Y%m%d"))
# soccom_bgc_argo_ds_profiles = soccom_bgc_argo_ds_profiles.assign(prof_HHMMSS   = pd.to_datetime(soccom_bgc_argo_ds_profiles.datetime.values).strftime("%H%M%S"))

In [89]:
soccom_bgc_argo_ds_profiles

<xarray.Dataset>
Dimensions:                (profile: 1448, pressure_interp: 100)
Coordinates:
  * profile                (profile) int64 12832 12835 12838 ... 752 27082 31810
    lat                    (profile) float64 52.73 52.72 52.53 ... -47.58 0.264
    lon                    (profile) float64 -32.02 -30.6 ... 106.1 -168.8
    datetime               (profile) datetime64[ns] 2014-03-01T19:29:42 ... 2...
  * pressure_interp        (pressure_interp) float64 20.0 40.0 ... 2e+03
Data variables: (12/14)
    CONFIG_MISSION_NUMBER  (profile) int32 10 13 16 21 7 28 ... 68 50 41 123 86
    CYCLE_NUMBER           (profile) int32 10 13 16 21 7 28 ... 68 50 41 123 86
    DATA_MODE              (profile) <U1 'D' 'D' 'D' 'D' 'D' ... 'A' 'A' 'A' 'A'
    DIRECTION              (profile) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (profile) int32 5904479 5904479 ... 5906032 5906300
    pressure               (profile, pressure_interp) float64 20.0 ... 2e+03
    ...                     ...
    depth                  (profile, pressure_interp) float64 19.82 ... 1.98e+03
    year                   (profile) int64 2014 2014 2014 ... 2022 2022 2022
    month                  (profile) int64 3 3 4 4 5 6 6 7 8 ... 9 9 9 9 9 9 9 9
    day                    (profile) int64 1 17 2 28 28 4 ... 14 15 16 17 20 26
    prof_YYYYMMDD          (profile) object '20140301' '20140317' ... '20220926'
    prof_HHMMSS            (profile) object '192942' '145025' ... '090142'
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

## Make changes for GMM code

In [82]:
# Calculate absolute salinity, conservative temperature, and potential density anomaly
profiles_antarctic = soccom_bgc_argo_ds_profiles
profiles_antarctic = profiles_antarctic.assign(prof_SA = gsw.conversions.SA_from_SP(SP = profiles_antarctic.practical_salinity,     p = profiles_antarctic.pressure, lon = profiles_antarctic.lon, lat = profiles_antarctic.lat))
profiles_antarctic = profiles_antarctic.assign(prof_CT = gsw.conversions.CT_from_t( SA = profiles_antarctic.prof_SA,  t = profiles_antarctic.temperature,   p = profiles_antarctic.pressure))
profiles_antarctic = profiles_antarctic.assign(sig0    = gsw.density.sigma0(        SA = profiles_antarctic.prof_SA, CT = profiles_antarctic.prof_CT))

# Limit depth range to <= 1000 dbar:
profiles_antarctic = profiles_antarctic.where(profiles_antarctic.pressure <= 1000, drop=True)

# Limit latitude to <= -30:
profiles_antarctic = profiles_antarctic.where(profiles_antarctic.lat <= -30, drop=True)



In [83]:
profiles_antarctic

<xarray.Dataset>
Dimensions:                (profile: 1236, pressure_interp: 50)
Coordinates:
  * profile                (profile) int64 10250 10268 12759 ... 32382 27082
    lat                    (profile) float64 -60.13 -56.93 ... -42.21 -47.58
    lon                    (profile) float64 176.0 -174.0 2.95 ... -78.04 106.1
    datetime               (profile) datetime64[ns] 2014-05-28T01:10:53 ... 2...
  * pressure_interp        (pressure_interp) float64 20.0 40.0 ... 980.0 1e+03
Data variables: (12/17)
    CONFIG_MISSION_NUMBER  (profile, pressure_interp) float64 7.0 7.0 ... 123.0
    CYCLE_NUMBER           (profile, pressure_interp) float64 7.0 7.0 ... 123.0
    DATA_MODE              (profile, pressure_interp) object 'D' 'D' ... 'A' 'A'
    DIRECTION              (profile, pressure_interp) object 'A' 'A' ... 'A' 'A'
    PLATFORM_NUMBER        (profile, pressure_interp) float64 5.904e+06 ... 5...
    pressure               (profile, pressure_interp) float64 20.0 ... 1e+03
    ...                     ...
    day                    (profile, pressure_interp) float64 28.0 28.0 ... 20.0
    prof_YYYYMMDD          (profile, pressure_interp) object '20140528' ... '...
    prof_HHMMSS            (profile, pressure_interp) object '011053' ... '15...
    prof_SA                (profile, pressure_interp) float64 34.04 ... 34.6
    prof_CT                (profile, pressure_interp) float64 3.987 ... 2.974
    sig0                   (profile, pressure_interp) float64 26.9 ... 27.44
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/29
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

In [90]:
profiles_antarctic.prof_SA.T

<xarray.DataArray 'prof_SA' (pressure_interp: 50, profile: 1236)>
array([[34.04234648, 34.12201226, 33.94848518, ..., 34.73217594,
        33.96548189, 34.35254599],
       [34.04223913, 34.12812776, 33.94801651, ..., 34.73622105,
        33.9702072 , 34.35154073],
       [34.0433078 , 34.12744323, 33.94799983, ..., 34.71612477,
        33.99567468, 34.35157772],
       ...,
       [34.73788444, 34.69197962, 34.73614403, ..., 34.47070338,
        34.48022076, 34.57258866],
       [34.74339803, 34.70532997, 34.74299011, ..., 34.47286567,
        34.50146453, 34.58920773],
       [34.74868807, 34.7126023 , 34.7585483 , ..., 34.47398827,
        34.51362684, 34.60071967]])
Coordinates:
  * profile          (profile) int64 10250 10268 12759 ... 32313 32382 27082
    lat              (profile) float64 -60.13 -56.93 -49.06 ... -42.21 -47.58
    lon              (profile) float64 176.0 -174.0 2.95 ... -143.6 -78.04 106.1
    datetime         (profile) datetime64[ns] 2014-05-28T01:10:53 ... 2022-09...
  * pressure_interp  (pressure_interp) float64 20.0 40.0 60.0 ... 980.0 1e+03
Attributes:
    casted:   1

In [ ]:
# for depth, can drop profile dimension
# need to deal with any NaNs in profile data -- GMM can't take NaNs
# select depth range, get rid of any profiles that are missing any value within that depth range (eg: 0-1000 = no data on the shelf)
# missing values in the middle can be interpolated
# any NaN values at top or bottom of depth range, no clear way to interpolate/extrapolate. IF replace value with mean value it looks wierd 
# 

## Save SOCCOM BGC-Argo data

In [84]:
savedate = date.today().strftime('%Y%m%d')

# Save dataset with formatting for GMM code:
profiles_antarctic.to_netcdf(save_directory + 'soccom_bgc_argo_ds_-180to180lon_-85to-30lat_20to1000depth.nc'.format('.nc'))
profiles_antarctic.to_netcdf(save_directory + 'soccom_bgc_argo_ds_-180to180lon_-85to-30lat_20to1000depth_' + savedate + '.nc'.format('.nc'))

# Save original SOCCOM BGC Argo dataset:
soccom_bgc_argo_ds_profiles.to_netcdf(save_directory + 'soccom_bgc_argo_ds.nc'.format('.nc'))
soccom_bgc_argo_ds_profiles.to_netcdf(save_directory + 'soccom_bgc_argo_ds_' + savedate + '.nc'.format('.nc'))

In [40]:
soccom_bgc_argo_ds_profiles

<xarray.Dataset>
Dimensions:                (N_PROF: 33494, N_LEVELS: 991, year: 33494,
                            month: 33494)
Coordinates:
  * N_PROF                 (N_PROF) int64 752 753 754 755 ... 31579 32075 28486
  * N_LEVELS               (N_LEVELS) int64 0 1 2 3 4 5 ... 986 987 988 989 990
    LATITUDE               (N_PROF) float64 22.87 22.9 22.93 ... -32.65 4.176
    LONGITUDE              (N_PROF) float64 -158.0 -158.0 ... -155.8 -119.2
    TIME                   (N_PROF) datetime64[ns] 2007-12-23T10:58:56 ... 20...
  * year                   (year) int64 2007 2007 2007 2007 ... 2022 2022 2022
  * month                  (month) int64 12 12 12 12 12 12 12 ... 9 9 9 9 9 9 9
Data variables: (12/13)
    CONFIG_MISSION_NUMBER  (N_PROF) int32 1 1 1 1 1 1 1 ... 143 62 54 69 63 209
    CYCLE_NUMBER           (N_PROF) int32 1 2 3 4 5 6 7 ... 143 62 54 69 63 209
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'D' 'D' ... 'A' 'A' 'A' 'A'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int32 5901468 5901468 ... 5906304 5906044
    POSITION_QC            (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ...                     ...
    PRES_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    PSAL                   (N_PROF, N_LEVELS) float64 34.99 34.99 ... nan nan
    PSAL_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TEMP                   (N_PROF, N_LEVELS) float64 24.47 24.47 ... nan nan
    TEMP_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    TIME_QC                (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://erddap.ifremer.fr/erddap
    Fetched_by:           hannah
    Fetched_date:         2022/09/23
    Fetched_constraints:  phy;WMO1901378;WMO1901379;WMO1902303;WMO1902304;WMO...
    Fetched_uri:          ['https://erddap.ifremer.fr/erddap/tabledap/ArgoFlo...
    history:              Variables filtered according to DATA_MODE; Variable...

In [50]:
np.min(soccom_bgc_argo_ds_profiles.LATITUDE)

<xarray.DataArray 'LATITUDE' ()>
array(-74.052)

In [48]:
np.max(soccom_bgc_argo_ds_profiles.LATITUDE)

<xarray.DataArray 'LATITUDE' ()>
array(61.07)

In [51]:
np.min(soccom_bgc_argo_ds_profiles.LONGITUDE)

<xarray.DataArray 'LONGITUDE' ()>
array(-178.874)

In [52]:
np.max(soccom_bgc_argo_ds_profiles.LONGITUDE)

<xarray.DataArray 'LONGITUDE' ()>
array(179.946)

In [ ]:
1901378_HRQC.nc

In [93]:
from argopy import DataFetcher
f = DataFetcher(src='gdac', ds='bgc', mode='expert', ftp='/Users/hannah/Documents/UW-PMEL/Research/so_co2_flux_repo/data/01_raw/SOCCOM_bgc_argo_float_data/SOCCOM_HiResQC_LIAR_21Dec2021_netcdf/').float(1901378)
f.load().data

FtpPathError: This path is not GDAC compliant (no `dac` folder with legitimate sub-folder):
/Users/hannah/Documents/UW-PMEL/Research/so_co2_flux_repo/data/01_raw/SOCCOM_bgc_argo_float_data/SOCCOM_HiResQC_LIAR_21Dec2021_netcdf/

In [95]:
import argopy
from argopy import DataFetcher as ArgoDataFetcher

In [96]:
ftproot, flist = argopy.tutorial.open_dataset('localftp')

In [97]:
argopy.set_options(local_ftp=ftproot)

In [ ]:
from argopy import DataFetcher
f = DataFetcher(src='gdac', ds='bgc', mode='expert', ftp='/Users/hannah/Documents/UW-PMEL/Research/so_co2_flux_repo/data/01_raw/SOCCOM_bgc_argo_float_data/SOCCOM_HiResQC_LIAR_21Dec2021_netcdf/').float(1901378)
f.load().data

In [98]:
with argopy.set_options(mode='expert'):
    ds = ArgoDataFetcher(src='localftp').profile(6901929, 2).to_xarray()
    print(ds.data_vars)

/Users/hannah/opt/miniconda3/envs/SO_BGC_clusters_venv/lib/python3.10/site-packages/argopy/fetchers.py:325: DeprecationWarning: Call to deprecated function __init__ (The 'localftp' data source is deprecated. It's been replaced by 'gdac'. It will raise an error after argopy 0.1.12).
  self.fetcher = self.Fetchers["float"](WMO=wmo, **self.fetcher_options)


NetCDF4FileNotFoundError: "Couldn't find NetCDF4 file: /Users/hannah/.argopy_tutorial_data/ftp/dac/*/1901378/1901378_prof.nc"